In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
X_train = pd.read_csv("X_train.csv")
X_train = X_train.iloc[:, 1:]
X_test = pd.read_csv("X_test.csv")
X_test = X_test.iloc[:, 1:]
Y_train = pd.read_csv("y_train.csv")
Y_test = pd.read_csv("y_test.csv")
Y_train = Y_train['Rating as Factor'].astype('category') #factorize trainset
Y_test = Y_test['Rating as Factor'].astype('category')   #factorize testset

In [3]:
def LogReg(X_train, Y_train):
    '''
    X_train: Training Set of X values
    Y_train: Training Set of Y values(factorized)
    '''
    lor = LogisticRegression(max_iter=100, tol=0.001,random_state=1, n_jobs=-1,solver='saga',warm_start=True) #increasing iterations to 1000 increases score by only 1% -> it is not worth the additional time

    pipe = Pipeline([('scaler', StandardScaler()), 
                     ('logreg', lor)])

    param_grid = {'logreg__penalty': ['elasticnet'], #elastic nets combines l1&l2
                  'logreg__C':[6,6.5,7,7.5,8],
                  'logreg__l1_ratio':[0,0.05,0.1,0.15,0.2,1]} #if 0, or 1 then l2 or l1 would be best. If between then the combination of both

    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
    grid = grid.fit(X_train,Y_train)
    
    return(grid)

In [4]:
import datetime
print(datetime.datetime.now())
grid = LogReg(X_train,Y_train)
print('Best parameters:', grid.best_params_) #best parameters are C=7 & ratio=0 -> l2 penalty function
print('Best CV accuracy:', grid.best_score_)
print('Test score:', grid.score(X_test,Y_test)) #31%
print(datetime.datetime.now()) #10 minutes

2020-04-04 13:28:56.635867
Best parameters: {'logreg__C': 7, 'logreg__l1_ratio': 0, 'logreg__penalty': 'elasticnet'}
Best CV accuracy: 0.3027588539829543
Test score: 0.31009860591635496
2020-04-04 13:39:14.746389


D:\Programme\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [5]:
# Predict classes
y_pred = grid.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 

Predicted   0    1   2   3    4    5   6    7    8   11  12  13  14  15  16
True                                                                       
0          234   36   9   0  230   13   0   68  137   0   0   0   0   4   0
1           62  135  15   0   66   19   1   16   73   0   0   0   0   2   0
2           89   21  16   0   20    4   2    4   42   0   0   0   0   0   0
3           17   26   4  11    0    0   0    0    1   0   0   0   0   0   0
4           94   51   3   0  636   46   2   38  183   0   1  18   6  13   0
5           65   25   9   1  290  104   3   16   97   0   0  16  11   5   0
6           12   19   1   6   20    0  15    9   16   0   0   0   0   0   0
7           84   32   0   0  294    6   0  206  150   0   0   2   1   1   0
8           73   49   8   0  390   35   0   72  285   0   1   7  10   5   1
9           22    0   0   0    0    0   0   10    8   0   0   0   0   0   0
10           0    0   0   0    3    0   0    0    0   0   0   7   0   0   0
11          